In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from random import uniform, randint, shuffle
from NPSmethods2 import readInImages, pupilFunc, getFreq
from numpy.fft import ifftshift, irfft2
import pickle as pkl
import matplotlib.cm as cm
from komm import AWGNChannel
from sklearn.preprocessing import QuantileTransformer, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
#Geranate random assortment of parameters
def make_Paras(seedParasMin, seedParasMax, numToGen):
    outPara = np.empty(shape =(numToGen, len(seedParasMin)))
    paraTemp = [None] * len(seedParasMin)
    for i in range(numToGen):
        for j in range(len(seedParasMin)):
            if j == 5:
                if seedParasMin[j] < 0:
                    low = 1.3
                else:
                    low = 0.77
                if seedParasMax[j] < 0:
                    high = 0.77
                else:
                    high = 1.3
            else:
                low = 1
                high = 1
            para = uniform(seedParasMin[j] * low, seedParasMax[j] * high)
            paraTemp[j] = para
        outPara[i] = paraTemp
    return outPara
        

#Generate M2k_Fit
def make_M2k_Fit(paras, imgSysData):
    
    A, tau, S0, alpha, phi, beta, delta_s = paras
    _, _, K_X, K_Y = getFreq(imgSysData["CCDPixelSize"],
                                      imgSysData["magnification"], (100,100))
    d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"])
    R_p, Theta_p = np.abs(K_X + 1j*K_Y) * d, np.angle(K_X + 1j*K_Y)
    p1 = pupilFunc(R_p, Theta_p + np.pi, tau, S0, alpha, phi, beta)
    p2 = np.conj(pupilFunc(R_p, Theta_p, tau, S0, alpha, phi, beta)) * \
            np.exp(-2*1j*delta_s)
    PSF = (p1 + p2) / (2 * np.cos(delta_s))
    M2k = np.abs(PSF)**2
    M2k_Fit = A * M2k
    M2k_Fit[M2k_Fit.shape[0]//2, M2k_Fit.shape[1]//2] = 0
    
    return M2k_Fit

#Function that gets largest values in an array
def Get_Max(array):
    max1 = np.amax(array) * 3
    max2 = np.amax(array[array < max1]) * 3
    array[np.where(array >= max2)] = np.mean(array)
    
    return array, max1, max2

#Function that mirrors an array with dimensions of odd values
def Mirror(array):
    array2 = np.flip(np.flip(array, 0), 1)
    array2 = np.delete(array2, (0), axis=1)
    newarray = np.concatenate((array, array2), 1)
    
    return newarray

#Turn one NPS to one OD
def NPSavg_To_Single(NPSavgs, numNPSs):
    NPSs = []
    NPSnum = 0
    
    #Reduce dimensions down to 49x49 image
    NPSavgs = np.delete(NPSavgs, np.s_[75:100], axis = 1)
    NPSavgs = np.delete(NPSavgs, np.s_[0:26], axis = 1)
    NPSavgs = np.delete(NPSavgs, np.s_[75:100], axis = 2)
    NPSavgs = np.delete(NPSavgs, np.s_[0:26], axis = 2)
    
    for NPSavg in NPSavgs:
        NPSnum += 1
        print("Parameters Set: %d" % NPSnum)
        for i in range(numNPSs):
            NPStemp = NPSavg.copy()
            Modifier = np.random.randint(9, 14)/10 #Modifier for noise-to-siganl ratio in center circle
            NSR = np.random.exponential(scale=Modifier, size=(49,25)) - 1 #Noise to signal ratio
            NSR, max1, max2 = Get_Max(NSR) #Get the maximum values for noise to signal ratio
            NSR = Mirror(NSR) #Mirror the noise to signal ratio
            Bkg = np.random.exponential(scale=0.032, size=(49,25)) #Generate background noise
            Bkg = Mirror(Bkg) #Mirror the background noise
            
            randc = randint(1,3) // 3 #Randomly select brightness of pixels that make a plus sign around center (noise feature)
            randx = randint(1,3) // 2 #Randomly determine whether the image has a horizontal line noise feature
            randy = randint(1,3) // 2 #Randomly determine whether the image has a vertical line noise feature
            
            mean = np.mean(NPStemp) #Mean value for the array used in creating noise features
            amax = np.amax(NPStemp) #Maximum value for the array used in creating noise features
            
            for j in range(len(NPStemp)):
                for k in range(len(NPStemp.T)):
                    NPStemp[j][k] += Bkg[j][k] #Add background noise
                    if (np.sqrt((24-j)**2 + (24-k)**2) < 20): #Add noise to center circle of image
                        if ((j == 23 or j == 25) and k == 24): #Add center bright points
                            if randc == 1:
                                NPStemp[j][k] = mean + mean * (max2) 
                            else :
                                NPStemp[j][k] = amax + amax * (max1)
                        elif (j == 24 and (k == 23 or k == 25)): #Add center bright points
                            if randc == 1:
                                NPStemp[j][k] = amax + amax * (max1)
                            else:
                                NPStemp[j][k] = mean + mean * (max2)
                        else: #Add noise scaled by noise to signal ratio
                            NPStemp[j][k] += NPStemp[j][k] * NSR[j][k] 
                    if j == 24: #Add horizontal and vertial lines
                        NPStemp[j][k] += .5 * mean * randx 
                    if k == 24:
                        NPStemp[j][k] += .5 * mean * randy
            NPStemp[24][24] = 0 #Zero the center dot
            NPStemp[NPStemp < 0] = 0 #Floor negative values to 0
            NPSs.append(NPStemp)
            
    return NPSs

In [ ]:
savepath = './results/' #Savepath for images and parameters

numNPSs = 5 #Number of NPS images to make for each parameter set
numParas = 20000 #Number of parameter sets to generate

#Import minimum and maximum seed parameters
seedParasMax = pkl.load(open(savepath + "seedParasMax.pkl", "rb"))
seedParasMin = pkl.load(open(savepath + "seedParasMin.pkl", "rb"))

NPSavgs = []
fakeNPSs = []
savedParas = []

imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
} 

#Create image and parameter arrays
for paras in make_Paras(seedParasMin, seedParasMax, numParas):
    NPSavgs.append(make_M2k_Fit(paras, imgSysData))
    for j in range(numNPSs):
        savedParas.append(paras)
fakeNPSs = NPSavg_To_Single(NPSavgs, numNPSs)

In [ ]:
#Save fake average images
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (100,100))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, NPSavgs[imnum], cmap=cm.jet, vmin=0, vmax=.5)
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPSavg_ims\\fake_NPSavg_" + str(imnum + 1))
    plt.close(fig)

In [ ]:
#Save fake single-OD images
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (49,49))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, fakeNPSs[imnum], cmap=cm.jet, vmin=0, vmax=.5)
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPS_ims\\fake_NPS_" + str(imnum + 1))
    plt.close(fig)

In [ ]:
#Save images and parameters
pkl.dump(NPSavgs, open(savepath + "fakeNPSavgs", "wb"))
pkl.dump(fakeNPSs, open(savepath + "fakeNPSs", "wb"))
pkl.dump(savedParas, open(savepath + "savedParas", "wb"))

In [ ]:
#Delete extreme outlier images
fakeNPSs = np.array(fakeNPSs).reshape((len(fakeNPSs),-1))
savedParas = np.array(savedParas)
savedParas = np.delete(savedParas, np.where(np.amax(fakeNPSs, axis=1) > 20), 0)
fakeNPSs = np.delete(fakeNPSs, np.where(np.amax(fakeNPSs, axis=1) > 20), 0)
savedParas = np.delete(savedParas, np.where(np.amin(fakeNPSs, axis=1) < 0), 0)
fakeNPSs = np.delete(fakeNPSs, np.where(np.amin(fakeNPSs, axis=1) < 0), 0)

#Normalize images and split into training and validation data
fakeNPSs /= 20
defocus = savedParas[:,[5]]
otherparas = np.delete(savedParas, 5, axis=1)
trainin, testin, trainout, testout, otherparasin, otherparasout = train_test_split(fakeNPSs, defocus, otherparas, test_size=0.25, shuffle=True)
for i in range(len(trainin)):
    trainin[i] -= np.mean(trainin[i])
    trainin[i] /= np.std(trainin[i])
for i in range(len(testin)):
    testin[i] -= np.mean(testin[i])
    testin[i] /= np.std(testin[i])
scaler = RobustScaler(unit_variance=True)
yscaler = scaler.fit(trainout)
pkl.dump(yscaler, open(savepath + "yscaler", "wb"))
trainout = yscaler.transform(trainout)
testout = yscaler.transform(testout)

In [ ]:
#Check number of samples. Should be close to numNPSs*numParas
print(np.shape(fakeNPSs))

In [ ]:
#Convert training and validation data to tensors
trainin = tf.convert_to_tensor(trainin)
trainout = tf.convert_to_tensor(trainout)
testin = tf.convert_to_tensor(testin)
testout = tf.convert_to_tensor(testout)

In [ ]:
#Store training and testing data
pkl.dump(trainin, open(savepath + "trainin", "wb"))
pkl.dump(trainout, open(savepath + "trainout", "wb"))
pkl.dump(testin, open(savepath + "testin", "wb"))
pkl.dump(testout, open(savepath + "testout", "wb"))
pkl.dump(otherparasin, open(savepath + "otherparasin", "wb"))
pkl.dump(otherparasout, open(savepath + "otherparasout", "wb"))

In [ ]:
#Create images with same parameters as a real image set
NPSavgs = []
paras = []
for i in range(100):
    paras.append([0.3307113143177933, 1.0000000000000002, 0.009999999999999986, 2.4599999999999995, -1.926865182387927, 0.3640009284887366, -0.03788728477733443])
for para in paras:
    NPSavgs.append(make_M2k_Fit(para, imgSysData))
fakeNPSs = NPSavg_To_Single(NPSavgs, numNPSs)

In [ ]:
#Save fake avg nps images with same parameters as real image set
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (100,100))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, NPSavgs[imnum], cmap=cm.jet, vmin=0, vmax=.5)
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPSavg_ims_SamePara\\fake_NPSavg_" + str(imnum + 1))
    plt.close(fig)

In [ ]:
#Save fake single-OD NPS images with same parameters as real image set
K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (49,49))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, fakeNPSs[imnum], cmap=cm.jet, vmin = 0, vmax=.5)
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPS_ims_SamePara\\fake_NPS_" + str(imnum + 1))
    plt.close(fig)

#### 